# Connectivity - Distance Between Habitats 3.3

### Import libraries and set up environment

In [10]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [2]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [ ]:
# Start with MPA_30km_habitat_int_singl_big_Replic2_3 (created in the EC_Replication_Habitats_2.3 script)
# That layer has already intersection the habitat MPA layer with the habitat layer, turned them to single-part, calculated the area, and removed chunks less than .24 km2
# Rename that layer to MPA_30km_habitat_int_singl_big_Conn3_3

in_feature = "MPA_30km_habitat_int_singl_big_Replic2_3"
out_feature = "MPA_30km_habitat_int_singl_big_Conn3_3"

arcpy.conversion.ExportFeatures(
    in_features= in_feature,
    out_features= out_feature,
    field_mapping=None,
    sort_field=None
)



In [42]:
# Make a list containing all the habitat types
habitat_types = ["deep_hard",
            "deep_soft",
           "shelf_hard",
           "shelf_soft",
           "slope_hard",
           "slope_soft",
           "shallow_hard",
           "shallow_soft",
           "coral",
           "mangrove",
           "saltmarsh",
           "seagrass",
           "seamount_knoll"
]

# Make a list to hold the results
results_list = []

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
in_feature = "MPA_30km_habitat_int_singl_big_Conn3_3"

def habitat_connectivity():
    for habitat in habitat_types:

        # Make feature layer for each habitat type:
        habitat_layer = arcpy.conversion.ExportFeatures(
            in_features= in_feature,
            out_features= output_gdb + "\\MPA_30km_habitat_int_singl_big_Conn3_3_" + habitat,
            where_clause="Habitat = '{}'".format(habitat)
        )
        
        print(habitat_layer)

        # Project the layer to the custom Azimuthal Equidistant (world)_carib projection:
        habitat_layer_azi = arcpy.management.Project(
            in_dataset= habitat_layer,
            out_dataset= str(habitat_layer) + "_Azi",
            out_coor_system='PROJCS["Azimuthal Equidistant (world)_carib",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-80.0],PARAMETER["Latitude_Of_Origin",20.0],UNIT["Meter",1.0]]',
            transform_method=None,
            in_coor_system='PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]',
            preserve_shape="NO_PRESERVE_SHAPE",
            max_deviation=None,
            vertical="NO_VERTICAL"
        )

        #buffer each habitat layer by 40 km
        habitat_layer_azi_buff = arcpy.analysis.PairwiseBuffer(
            in_features= habitat_layer_azi,
            out_feature_class= str(habitat_layer_azi) + "_40buff",
            buffer_distance_or_field="40 Kilometers",
            dissolve_option="NONE",
            dissolve_field=None,
            method="PLANAR",
            max_deviation="0 Meters"
        )

        #intersect the buffered MPAs
        habitat_layer_azi_buff_int = arcpy.analysis.PairwiseIntersect(
            in_features= habitat_layer_azi_buff,
            out_feature_class= str(habitat_layer_azi_buff) + "_int",
            join_attributes="ALL",
            cluster_tolerance=None,
            output_type="INPUT"
        )
        
        
        # Make layers of the connected and not connected MPAs by re-matching the intersecting buffers with their corresponding MPA footprints
        
        #select the intersecting buffers
        selection1 = arcpy.management.SelectLayerByLocation(
            in_layer= habitat_layer_azi_buff,
            overlap_type="INTERSECT",
            select_features= habitat_layer_azi_buff_int,
            search_distance=None,
            selection_type="NEW_SELECTION",
            invert_spatial_relationship="NOT_INVERT"
        )
        
        selected_count = int(arcpy.GetCount_management(selection1).getOutput(0))
        print(f"Number of selected features in {selection1}: {selected_count}")

        # Convert those back to their corresponing MPA footprints by selecting the MPAs that are completely within the selected buffers
        selection2 = arcpy.management.SelectLayerByLocation(
            in_layer= habitat_layer_azi,
            overlap_type="COMPLETELY_WITHIN",
            select_features= selection1,
            search_distance=None,
            selection_type="ADD_TO_SELECTION",
            invert_spatial_relationship="NOT_INVERT"
        )
        
        selected_count = int(arcpy.GetCount_management(selection2).getOutput(0))
        print(f"Number of connected features for {habitat}: {selected_count}")

        # Extract just the feature class name from the full path so I can contantonate it below
        habitat_layer_azi_name = os.path.basename(habitat_layer_azi.getOutput(0))

        # make a new layer of the connected MPAs
        habitat_layer_azi_conn = arcpy.conversion.FeatureClassToFeatureClass(
            in_features=selection2,
            out_path=output_gdb,
            out_name= habitat_layer_azi_name + "_Conn",
            where_clause="",
        )
        
        #select the inverse of selection2 to make a layer of not connected MPAs
        selection2_inverse = arcpy.management.SelectLayerByLocation(
            in_layer= habitat_layer_azi,
            overlap_type="COMPLETELY_WITHIN",
            select_features= selection1,
            search_distance=None,
            selection_type="ADD_TO_SELECTION",
            invert_spatial_relationship="INVERT"
        )

        selected_count = int(arcpy.GetCount_management(selection2_inverse).getOutput(0))
        print(f"Number of NOT connected features for {habitat}: {selected_count}")
        
        # Extract just the feature class name from the full path so I can contantonate it below
        habitat_layer_azi_name = os.path.basename(habitat_layer_azi.getOutput(0))
            
        # make a new layer of the connected MPAs
        habitat_layer_azi_notconn = arcpy.conversion.FeatureClassToFeatureClass(
            in_features=selection2_inverse,
            out_path=output_gdb,
            out_name= habitat_layer_azi_name + "_NotConn",
            where_clause="",
        )
        
        # Get the counts
        habitat_count = int(arcpy.GetCount_management(habitat_layer_azi).getOutput(0))
        conn_count = int(arcpy.GetCount_management(habitat_layer_azi_conn).getOutput(0))
        notconn_count = int(arcpy.GetCount_management(habitat_layer_azi_notconn).getOutput(0))

        
        # append the data for this iteration to the list
        results_list.append({
            "Habitat": habitat,
            "Total number of MPA footprints": habitat_count,
            "Number of connected MPA footprints": conn_count,
            "Number of not connected MPA footprints": notconn_count
        })

habitat_connectivity()

# Create a DataFrame from the accumulated data
df = pd.DataFrame(results_list)

# Print and show the DataFrame
print(df)
print("\nDataFrame head:")
print(df.head())

F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb\MPA_30km_habitat_int_singl_big_Conn3_3_deep_hard
Number of selected features in MPA_30km_habitat_int_singl_b39: 33
Number of connected features for deep_hard: 33
Number of NOT connected features for deep_hard: 2
F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb\MPA_30km_habitat_int_singl_big_Conn3_3_deep_soft
Number of selected features in MPA_30km_habitat_int_singl_b42: 42
Number of connected features for deep_soft: 42
Number of NOT connected features for deep_soft: 7
F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb\MPA_30km_habitat_int_singl_big_Conn3_3_shelf_hard
Number of selected features in MPA_30km_habitat_int_singl_b45: 424
Number of connected features for shelf_hard: 424
Number of NOT connected features for shelf_hard: 11
F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb\MPA_30km_habitat_int_singl_big_Conn3_3_shelf_soft
Number of selected features 

In [43]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Connectivity"
table = df
output_name = "Habitat_Connectivity_3_3_results.xlsx"

table.to_excel(os.path.join(table_outputs_folder, output_name))

